# Defining a Model

This section demonstrates the definition of models.

As noted in `1_Getting_Started`, the general workflow enabled by the module is the following:
1. Instantiate a model object and define its components
2. Perform preprocessing operations
3. Define an analysis object
4. Run the analysis
5. Post-process the analysis results

This jupyter notebook demonstrates steps #1 and #2.

## Model Definition Overview

The overall procedure for defining the elements of a model can be broken down into the following steps:
- [ ] Set active section, placement, angle
- [ ] Set active levels and groups
- [ ] Define line elements
- [ ] Repeat until all line elements are defined
- [ ] Apply pre-processing methods

The model can be visualized at any step in the process to confirm its validity.

**To see all the available arguments of each of the following methods, please look at their source code and read the docstrings**.

In [ ]:
# imports
import sys
sys.path.append('../src')
import model

import numpy as np

In [ ]:
# Instantiate a model
b = model.Model()

In [ ]:
# Define levels
b.add_level('base', 0.00, 'fixed')
for i in range(3):
    b.add_level(str(i+1), 144.00*(i+1))

In [ ]:
# activate a material
b.set_active_material('steel-bilinear-fy50')

In [ ]:
# define line element sections
b.add_sections_from_json(
    "../section_data/sections.json",
    'W',
    ["W24X94"])
b.set_active_section("W24X94")

# note: the sections.json relative path
# depends on where the python file is located.

In [ ]:
# set active levels
b.set_active_levels("all_above_base")

In [ ]:
# assign a surface load of 1 lb / ft2
b.assign_surface_load(1.00)

In [ ]:
b.active_placement = 'centroid'

In [ ]:
p1 = np.array((0.00, 0.00))
p2 = np.array((360., 0.00))
p3 = np.array((360., 360.))
p4 = np.array((0.00, 360.00))

In [ ]:
for pt in [p1, p2, p3, p4]:
    b.add_column_at_point(
        pt,
        n_sub=4,
        geom_transf='Corotational')

In [ ]:
b.active_placement = 'top_center'

for pair in ((p1, p2), (p2, p3), (p3, p4), (p4, p1)):
    b.add_beam_at_points(
        pair[0], pair[1],
        n_sub=4)


In [ ]:
# assign a linearly distributed load of 20 lb/in to the perimeter beams
b.select_perimeter_beams_all()
b.selection.add_UDL(np.array((0.00, 0.00, -20.00)))


In [ ]:
# visualize the model
# showing the centroidal axes
b.plot_building_geometry()

In [ ]:
# extruded frame view
b.plot_building_geometry(extrude_frames=True)

## Preprocessing

Now that all the intended elements have been defined, we can apply pre-processing methods to the model.
Two very common methods are the following:
- `diaphragms` assigns rigid diaphragm constraints to all levels that contain beams
- `tributary_area_analysis` distributes loads from the floors to the supporting elements
- `self_weight_and_mass` assigns self-weight loads and lumped self-mass to all line elements

In [ ]:
import preprocess

In [ ]:
preprocess.diaphragms(b)
preprocess.tributary_area_analysis(b)
preprocess.self_weight_and_mass(b)

In [ ]:
b.plot_building_geometry()
b.plot_building_geometry(extrude_frames=True)

More preprocessing functions are available.

#### Steel moment frame panel zones

In [ ]:
import toy_structures

Here is a steel moment frame before preprocessing:

In [ ]:
cam1 = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0.0, y=0, z=-0.30),
    eye=dict(x=0.00, y=0.1, z=-0.30),
    projection={
        "type": "perspective"
    }
)
cam2 = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0.0, y=0, z=0.00),
    eye=dict(x=0.00, y=4.0, z=0.00),
    projection={
        "type": "perspective"
    }
)

toy_structures.smrf_3_4.plot_building_geometry(extrude_frames=False, global_axes=True, frame_axes=False, camera=cam1)
toy_structures.smrf_3_4.plot_building_geometry(extrude_frames=True, global_axes=False, camera=cam2)

In [ ]:
preprocess.model_steel_frame_panel_zones(toy_structures.smrf_3_4)

After preprocessing:

In [ ]:
toy_structures.smrf_3_4.plot_building_geometry(extrude_frames=False, global_axes=True, frame_axes=False, camera=cam1)
toy_structures.smrf_3_4.plot_building_geometry(extrude_frames=True, global_axes=False, camera=cam2)